In [5]:
import pandas as pd
import numpy as np

### Load some data and prepare for PU learning.
Each row corresponds to a material and each column is a feature from either elemental data or density functional theory calculations.
We discard non-numeric columns (e.g. compound labels) and rename the "Synt" column to "PU_label" so the PULearner knows where to find the labels of positive (1) or unlabeled (0).

In [29]:
df = pd.read_excel('../test_files/MAX_dataset.xlsx', index_col=0)
df = df.iloc[:, 4:]  # Get rid of non-numeric features
df = df.rename(index=str, columns={'Synt': 'PU_label'})
df.to_json('MAX_dataset.json')

df = pd.read_excel('../test_files/MX_dataset.xlsx', index_col=0)
df = df.iloc[:, 3:]  # Get rid of non-numeric features
df = df.rename(index=str, columns={'Synt': 'PU_label'})
df.to_json('MX_dataset.json')

In [3]:
from mlmsynth import PULearner

SyntaxError: invalid syntax (<ipython-input-3-dfb79cc55e6c>, line 1)